In [71]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [72]:
# for view dataframe after every change
df.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,0.034021,0.2,1,1,1,0.184475,1.850000e+09,61666.67
1,0.030928,0.2,1,1,1,0.184475,1.850000e+09,61666.67
2,0.050515,0.4,1,1,1,0.023941,5.500000e+08,18333.33
3,0.067010,0.4,1,1,1,0.027535,9.025000e+08,30083.33
4,0.095876,0.4,1,1,1,0.549636,7.000000e+09,233333.33


In [73]:
cdf = pd.read_csv("")
# take a look at the dataset
cdf.head(9)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33
5,70,2,True,True,False,North Program Organization,2.050000e+09,68333.33
6,87,2,True,True,True,Pardis,6.000000e+08,20000.00
7,59,1,True,True,True,Shahran,2.150000e+09,71666.67
8,54,2,True,True,False,Andisheh,4.930000e+08,16433.33


In [74]:
df=cdf[['Area','Room','Parking','Warehouse','Elevator','Address','Price','Price(USD)']]
df.head(9)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33
5,70,2,True,True,False,North Program Organization,2.050000e+09,68333.33
6,87,2,True,True,True,Pardis,6.000000e+08,20000.00
7,59,1,True,True,True,Shahran,2.150000e+09,71666.67
8,54,2,True,True,False,Andisheh,4.930000e+08,16433.33


In [75]:
# convert "Area" column to integer
df.loc[:,'Area'] = df['Area'].str.replace(',', '').str.strip().astype(int)
# find and show values greater than 3000
print(df[df['Area'] > 3000]['Area'])
# delete values greater than 3000
df = df[df['Area'] <= 3000]
df = df.reset_index(drop=True)

570      3310000000
709     16160000000
1604     8400000000
2171           3600
2802     2550000000
Name: Area, dtype: int64


In [76]:
# find rows with empty cells in "Address" column
empty_name_mask = df['Address'].isna()
# print row numbers with empty cells in "Address" column
empty_name_indices = df.index[empty_name_mask]
print("Rows with empty cells in 'Address' column:")
print(empty_name_indices)
# delete rows with empty cells in "Address" column
df = df[~empty_name_mask]
df = df.reset_index(drop=True)

Rows with empty cells in 'Address' column:
Int64Index([  43,  661,  705, 1106, 1107, 1575, 1793, 2068, 2069, 2124, 2239,
            2333, 2422, 2724, 3020, 3171, 3257, 3290, 3310, 3313, 3350, 3396,
            3429],
           dtype='int64')


In [77]:
# Change Address column with relative of Price and Area values
dic={}
for i in range (0,len(df['Address'])):
    if df['Address'][i] in dic.keys():
        a = df['Price(USD)'][i]/(df['Area'][i])
        b= (a+dic[df['Address'][i]])/2
        dic[df['Address'][i]]=b
        
    else:
        a = df['Price(USD)'][i]/(df['Area'][i])
        dic[df['Address'][i]]=a
for index, row in df.iterrows():
    address = row['Address']
    if address in dic:
        df.at[index, 'Address'] = dic[address]

# In case of scaling Area and Room 

In [78]:
# If we scale 2 columns Area and Room and change type of boolean columns
# Select the numeric columns
numeric = ['Area','Room','Address']
# Select boolean columns
boolean = ['Parking','Warehouse','Elevator']

In [79]:
# Normalize the numeric columns
scaler = MinMaxScaler()
df[numeric] = scaler.fit_transform(df[numeric])
# Convert the boolean columns to binary
df[boolean] = df[boolean].astype(int)

## If we don't want scale Numeric columns skip 2 above cells

In [80]:
# Creating train and test dataset
# I tested price and price(usd) in seperate function to learn the model and both of them are simple with use them in a single function
X = df.drop(['Price','Price(USD)'], axis=1)
y = df[['Price','Price(USD)']]
y=np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [81]:
X_train=np.asanyarray(X_train)
X_test=np.asanyarray(X_test)
y_train=np.asanyarray(y_train)
y_test=np.asanyarray(y_test)

In [82]:
#Creat a model
lr = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
lr.fit(X_train, y_train)
# make a prediction on the test set
y_pred = lr.predict(X_test)
# evaluate the performance of the model on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('R^2:', r2)

MSE: 0.18698123432397384
R^2: 0.8385530390699634


In [83]:
# define the new house features
new = {'Area': 79,
             'Room': 2,
             'Parking': True,
             'Warehouse': True,
             'Elevator': True,
             'Address': 'Shahran'}

new=pd.DataFrame(new, index=[0])

new['Address'][0]=dic[new['Address'][0]]
new[numeric] = scaler.transform(new[numeric])
new[boolean] = new[boolean].astype(int)
new=np.asanyarray(new)
predicted_log = lr.predict(new)
predicted = np.exp(predicted_log)
print('Price in Toman is : ',predicted[0][0] ,'and Price in Usd is :' ,predicted[0][1])

Price in Toman is :  2352686439.661199 and Price in Usd is : 78422.88141440648


/var/folders/cy/v8prby6d1sn2xp04c_y3vhqm0000gn/T/ipykernel_61505/3037719919.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Address'][0]=dic[new['Address'][0]]
